# Clustering de documents

## Imports

In [2]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

In [3]:
import nltk

nltk.download('punkt')
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\taleb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\taleb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
data_path = "../data/txt/"

## Choisir une décennie

In [5]:
DECADE = '1950'

## Charger tous les  fichiers de la décennie et en créer une liste de textes

In [6]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [7]:
# Exemple de fichiers
files[:5]

['KB_JB838_1950-01-06_01-00002.txt',
 'KB_JB838_1950-01-07_01-00013.txt',
 'KB_JB838_1950-01-08_01-00011.txt',
 'KB_JB838_1950-01-10_01-00006.txt',
 'KB_JB838_1950-01-15_01-00010.txt']

In [8]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [9]:
# Exemple de textes
texts[0][:400]

'/ LE SOIR Une importante manifestation franco-belge Le Comité franco-belge du Commerce, de l’Industrie et de l’Agriculture qui. au cours de l’année écoulée et dans les circonstances les plus difficiles a rendu aux relations économiques entre les deux pays des services considérables, donnera le 12 janvier, à 19 h. 30, un dîner à l’occasion de l’anniversaire de son installation. Ce comité, qui fonct'

## Vectoriser les documents à l'aide de TF-IDF

In [10]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [11]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [12]:
tfidf_vectors = vectorizer.fit_transform(texts)

c:\Users\taleb\Desktop\ULB\STIC-B545 - Traitement automatique de corpus\TP\tac\tac_venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
# Détail de la matrice
tfidf_vectors

<1000x3506 sparse matrix of type '<class 'numpy.float64'>'
	with 703537 stored elements in Compressed Sparse Row format>

### Imprimer le vecteur tf-IDF du premier document

In [14]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

parti           0.276938
loyer           0.152177
janvier         0.148006
populaire       0.147153
propriétaire    0.143666
                  ...   
habitants       0.000000
agit            0.000000
haecht          0.000000
hainaut         0.000000
#               0.000000
Length: 3506, dtype: float64

## Comprendre les vecteurs et leurs "distances"

In [15]:
cosine([1, 2, 3], [1, 2, 3])

0

In [16]:
cosine([1, 2, 3], [1, 2, 2])

0.02004211298777725

In [17]:
cosine([1, 2, 3], [2, 2, 2])

0.07417990022744858

### Tests sur nos documents

In [18]:
tfidf_array = tfidf_vectors.toarray()

In [19]:
# Vecteur du document 0
tfidf_array[0]

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.03783743])

In [20]:
# Vecteur du document 1
tfidf_array[1]

array([0.       , 0.0067047, 0.       , ..., 0.       , 0.       ,
       0.       ])

In [21]:
cosine(tfidf_array[0], tfidf_array[1])

0.9314339868905285

## Appliquer un algorithme de clustering sur les vecteurs TF-IDF des documents

Pour en savoir plus sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

### Définir un nombre de clusters

In [22]:
N_CLUSTERS = 5

### Instancier le modèle K-Means et ses arguments

In [23]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [24]:
clusters = km_model.fit_predict(tfidf_vectors)

In [25]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [26]:
pprint(dict(clustering))

{0: ['KB_JB838_1950-01-08_01-00011.txt',
     'KB_JB838_1950-01-22_01-00014.txt',
     'KB_JB838_1950-02-15_01-00011.txt',
     'KB_JB838_1950-02-15_01-00014.txt',
     'KB_JB838_1950-02-16_01-00014.txt',
     'KB_JB838_1950-02-25_01-00013.txt',
     'KB_JB838_1950-03-29_01-00008.txt',
     'KB_JB838_1950-04-21_01-00010.txt',
     'KB_JB838_1950-04-25_01-00011.txt',
     'KB_JB838_1950-05-20_01-00016.txt',
     'KB_JB838_1950-06-04_01-00012.txt',
     'KB_JB838_1950-06-04_01-00016.txt',
     'KB_JB838_1950-06-11_01-00015.txt',
     'KB_JB838_1950-06-24_01-00011.txt',
     'KB_JB838_1950-06-25_01-00013.txt',
     'KB_JB838_1950-07-08_01-00009.txt',
     'KB_JB838_1950-07-20_01-00010.txt',
     'KB_JB838_1950-09-08_01-00012.txt',
     'KB_JB838_1950-09-09_01-00011.txt',
     'KB_JB838_1950-09-21_01-00009.txt',
     'KB_JB838_1950-10-05_01-00012.txt',
     'KB_JB838_1950-10-15_01-00016.txt',
     'KB_JB838_1950-10-15_01-00018.txt',
     'KB_JB838_1950-10-27_01-00014.txt',
     'KB_JB838_1

## Visualiser les clusters

### Réduire les vecteurs à 2 dimensions à l'aide de l'algorithme PCA
Cette étape est nécessaire afin de visualiser les documents dans un espace 2D

https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales

In [26]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [27]:
reduced_vectors[:10]

array([[-0.32580699,  0.11404385],
       [ 0.11344714, -0.30211052],
       [ 0.44352734, -0.25255157],
       [-0.23898552, -0.03761837],
       [-0.05189779, -0.10505036],
       [ 0.61297815,  0.50406052],
       [-0.2041063 , -0.03508382],
       [-0.19454707, -0.03815659],
       [ 0.62392888,  0.4558867 ],
       [ 0.15642299, -0.01433006]])

### Générer le plot

In [29]:
import matplotlib
matplotlib.use('Agg')  # backend non interactif
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.scatter(x_axis, y_axis, s=100, c=clusters)
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.title("Clustering des documents")
plt.savefig("clusters.png")  # Sauvegarde en fichier PNG
plt.show()


C:\Users\taleb\AppData\Local\Temp\ipykernel_20680\990718850.py:11: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
